<a href="https://www.kaggle.com/code/vanha2301/voiceover-dubbing?scriptVersionId=282800939" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Voice

## Cài đặt thư viện và Tải video mẫu

In [1]:
# 1. Cài đặt thư viện faster-whisper (phiên bản chạy nhanh trên GPU)
!pip install faster-whisper


print("✅ Đã cài đặt xong và tải video mẫu thành công!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 48.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.8/37.8 MB 54.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 95.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00
✅ Đã cài đặt xong và tải video mẫu thành công!


## Code xử lý (Core Logic)

In [2]:
import time
from faster_whisper import WhisperModel

# --- CẤU HÌNH ---
video_path = "/kaggle/input/example-vod/KhiAnhChayVePhiaEm.mp4" # Đường dẫn video (thay bằng file của bạn sau này)
output_file = "/kaggle/working/ex.txt"
model_size = "large-v3"         # Mô hình xịn nhất
# model_size = "medium"         # Dùng cái này nếu muốn test nhanh hơn nữa

print(f"🚀 Đang load mô hình {model_size} vào GPU T4...")
# device="cuda" -> Bắt buộc để dùng GPU
# compute_type="float16" -> Tận dụng sức mạnh tính toán của T4
model = WhisperModel(model_size, device="cuda", compute_type="float16")

print("⏳ Đang bắt đầu xử lý video (Transcribing)...")
start_time = time.time()

# --- XỬ LÝ ---
# beam_size=5: Giúp AI dò tìm câu chính xác hơn
segments, info = model.transcribe(
    video_path, 
    beam_size=5,
    vad_filter=True,
    vad_parameters=dict(min_silence_duration_ms=500)
)

# --- HIỂN THỊ KẾT QUẢ ---
print(f"\nDetected language: {info.language} (Độ tin cậy: {info.language_probability:.2f})")
print("-" * 50)
print(f"{'TIME START':<10} | {'TIME END':<10} | {'TEXT CONTENT'}")
print("-" * 50)

full_text_log = ""
print(video_path)
# segments là một generator (nó chạy đến đâu nhả chữ đến đó)
with open(output_file, "w", encoding="utf-8") as f:
    for segment in segments:
        # Định dạng thời gian cho đẹp
        start = f"{segment.start:.2f}s"
        end = f"{segment.end:.2f}s"
        text = segment.text
        
        # In ra màn hình dạng bảng
        print(start)
        f.write(start + "\n")
        print(end)
        f.write(end + "\n")
        print(text)
        f.write(text + "\n")
        
    
        # Lưu lại text để dùng cho việc khác (như Dịch thuật)
        full_text_log += text + " "

end_time = time.time()
print("-" * 50)
print(f"✅ Xử lý xong trong: {end_time - start_time:.2f} giây")
print("\n📝 TOÀN BỘ VĂN BẢN GỘP:")
print(full_text_log.strip())

🚀 Đang load mô hình large-v3 vào GPU T4...
⏳ Đang bắt đầu xử lý video (Transcribing)...

Detected language: zh (Độ tin cậy: 0.99)
--------------------------------------------------
TIME START | TIME END   | TEXT CONTENT
--------------------------------------------------
/kaggle/input/example-vod/KhiAnhChayVePhiaEm.mp4
1.10s
2.18s
你怎么来了
2.18s
4.03s
知道你是我依旧
4.03s
4.97s
所以我来
4.97s
6.21s
请你回家
--------------------------------------------------
✅ Xử lý xong trong: 1.76 giây

📝 TOÀN BỘ VĂN BẢN GỘP:
你怎么来了 知道你是我依旧 所以我来 请你回家


# Translate

In [3]:
print(full_text_log)

你怎么来了 知道你是我依旧 所以我来 请你回家 


In [4]:
# Cài đặt phiên bản protobuf "hòa bình" nhất cho cả TensorFlow và Transformers
!pip install protobuf==4.25.3

In [2]:
from faster_whisper import WhisperModel
from transformers import pipeline
import torch

# --- BƯỚC 1: KHỞI TẠO CÁC MÔ HÌNH ---

# 1.1 Load Whisper (Để nghe tiếng Trung)
print("🚀 Đang load Whisper Large-v3...")
whisper_model = WhisperModel("large-v3", device="cuda", compute_type="float16")

# 1.2 Load NLLB (Để dịch Trung -> Việt)
# zho_Hans = Tiếng Trung Giản Thể (Phổ biến nhất)
# vie_Latn = Tiếng Việt
print("🚀 Đang load NLLB Translator...")
translator = pipeline(
    'translation', 
    model="facebook/nllb-200-distilled-1.3B", 
    device=0, # Chạy trên GPU
    src_lang="zho_Hans", 
    tgt_lang="vie_Latn"
)

# --- BƯỚC 2: XỬ LÝ VIDEO ---

video_path = "/kaggle/input/example-vod/KhiAnhChayVePhiaEm.mp4" 

print("\n🎧 Đang tách lời tiếng Trung...")
# task="transcribe": Yêu cầu nó chép đúng tiếng gốc (Trung), đừng cố dịch sang Anh
segments, info = whisper_model.transcribe(
    video_path, 
    beam_size=5, 
    language="zh",     # Ép nhận diện tiếng Trung
    task="transcribe", # Chỉ chép lại, không dịch vội
    vad_filter=True
)

print(f"✅ Đã nhận diện ngôn ngữ: {info.language}")
print("="*60)
print(f"{'THỜI GIAN':<15} | {'TIẾNG TRUNG (GỐC)':<30} | {'TIẾNG VIỆT (DỊCH)'}")
print("="*60)

# --- BƯỚC 3: VỪA CHẠY VỪA DỊCH ---
output_file = "phu_de_viet.txt"

with open(output_file, "w", encoding="utf-8") as f:
    for segment in segments:
        text_trung = segment.text
        
        # Đưa tiếng Trung vào NLLB để dịch sang Việt
        # max_length=512 để tránh lỗi nếu câu quá dài
        ket_qua_dich = translator(text_trung, max_length=512)
        text_viet = ket_qua_dich[0]['translation_text']
        
        # In ra màn hình cho sướng mắt
        time_stamp = f"{segment.start:.2f}s -> {segment.end:.2f}s"
        print(f"{time_stamp:<15} | {text_trung[:30]:<30}... | {text_viet}")
        
        # Ghi vào file (chỉ ghi tiếng Việt hoặc cả hai tùy bạn)
        # Ở đây mình ghi dạng: [Thời gian] Tiếng Việt
        f.write(f"[{time_stamp}] {text_viet}\n")
        

print(f"\n✅ Xong! Đã lưu phụ đề tiếng Việt vào file: {output_file}")

2025-11-30 07:14:51.396206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764486891.610593      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764486891.675500      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0



🎧 Đang tách lời tiếng Trung...
✅ Đã nhận diện ngôn ngữ: zh
THỜI GIAN       | TIẾNG TRUNG (GỐC)              | TIẾNG VIỆT (DỊCH)
1.10s -> 2.18s  | 你怎么来了                         ... | Sao anh đến đây?
2.18s -> 4.10s  | 知道你是我女朋友                      ... | Biết rằng em là bạn gái của anh
4.10s -> 4.94s  | 所以我来                          ... | Đó là lý do tôi đến đây.
4.94s -> 6.20s  | 请你回家                          ... | Xin hãy về nhà.

✅ Xong! Đã lưu phụ đề tiếng Việt vào file: phu_de_viet.txt


# Speech

# Dowloading Lib

## Loading Model

In [6]:
# Cài trực tiếp từ GitHub để lấy bản vá lỗi mới nhất
!pip install --upgrade git+https://github.com/rany2/edge-tts.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/rany2/edge-tts.git to /tmp/pip-req-build-otk8eyiy
  Running command git clone --filter=blob:none --quiet https://github.com/rany2/edge-tts.git /tmp/pip-req-build-otk8eyiy
  Resolved https://github.com/rany2/edge-tts.git to commit 27d62488330aa9ff4487152a934ef2348c22dfeb
  Preparing metadata (setup.py) ... done
